# *GENERATE EXCELS FROM DISEASE SIMILARITY TABLE*

**Author: Lucía Prieto Santamaría **(lucia.prieto.santamaria@alumnos.upm.es)

This notebook was written to generate similarity matrices in .csv files from the table "similarity" in DISNET database (EDSSSDB). It generates as many excels as measures are selected.

In [1]:
# Import the needed libraries
import conexion_edsssdb # This is a module already scripted that makes connection automatized
import datetime # Module to print the exact time when a number of insertions in the csv is reached

In [2]:
# Connection object creation with connection variables. EDSSSDB is the database to connect to.
cnx = conexion_edsssdb.conectar('user', 'password', 'host', 'port', 'db')

Conectado a edsssdb


In [3]:
def read_cuis(cnx):
    """ Function that gets from EDSSSDB all the available CUIs from the biological layer"""
    
    # Query to the table "has_biological_info" to get all the CUIs
    qry =("SELECT DISTINCT cui"
           "  FROM has_biological_info"
           " ORDER BY cui;")

    crs = cnx.cursor()
    crs.execute(qry)
    
    # It returns the list with all the CUIs
    return crs.fetchall()

In [4]:
cuis = read_cuis(cnx)
maxcui = len(cuis)
# We print the number of CUIs that we have returned
print('Number of CUIs:', maxcui)

Number of CUIs: 3671


In [5]:
# Folder and group name to identify the correct directory
# IMPORTANT!!!! THE FOLDER NEEDS TO BE PREVIOUSLY CREATED BY THE USER
directory_name = 'total'

# We select the similarity metrics from which we want to generate the excels
sim_metrics = (
   'S_gen_cos',
   'S_gen_jaccard',
   'S_gen_dice',
   'S_prot_cos',
   'S_prot_jaccard',
   'S_prot_dice',
   'S_path_cos',
   'S_path_jaccard',
   'S_path_dice',
   'S_ppi_cos',
   'S_ppi_jaccard',
   'S_ppi_dice'
   'S_path_cos',
   'S_path_jaccard',
   'S_path_dice',
   'S_ppi_cos',
   'S_ppi_jaccard',
   'S_ppi_dice',
   'S_term_cos',
   'S_term_jaccard',
   'S_term_dice'
   )


# This variable will be used to concatenate all the metrics in a string in order to make the proper query
selmet = ""

# This list will contain the file objets created
files = []


for metric in sim_metrics:
     # We create a csv file for each metric
     files.append(open("excels/" + directory_name + metric + ".csv", "w"))
     # We keep concatenating the metrics in a string
     selmet = selmet + ', ' + metric


# We write the query to the table similarity
query = "SELECT id_dis1 AS id_cui1, id_dis2 AS id_cui2" + selmet + \
        "  FROM similarity" + \
        " UNION ALL " + \
        "SELECT id_dis2 AS id_cui1, id_dis1 AS id_cui2" + selmet + \
        "  FROM similarity" + \
        " ORDER BY id_cui1, id_cui2"

# We print the query in order to know that it is well-formatted     
print('Query to database:\n',query)

Query to database:
 SELECT id_dis1 AS id_cui1, id_dis2 AS id_cui2, S_gen_cos  FROM similarity UNION ALL SELECT id_dis2 AS id_cui1, id_dis1 AS id_cui2, S_gen_cos  FROM similarity ORDER BY id_cui1, id_cui2


In [6]:
# In this case, we use fetchone instead of fetchall
crs = cnx.cursor()
crs.execute(query)
row = crs.fetchone()

# We print the moment the proccess starts
print(datetime.datetime.now())

# We prepare the first row of the files
for file in files:
     file.write("CUI") # The first field will be 'CUI'
     
     for cui in cuis:
        file.write("," + cui[0]) # Writing the diseases CUIs in the fist row of each file.
     
     file.write("\n")


# This variable will be used to print the moment each time this variable reaches a number multiple of 1000000.
# It represents the number of fields that have been written
cont = 0 


for cuiy in cuis: # Getting through the disease for each row
    
    for file in files:
        file.write(cuiy[0]) # Printing the CUI in the first column of each row
    
    
    for cuix in cuis: # Getting through the disease for each column
        
        cont += 1 
        
        if cont % 1000000 == 0:
            # Printing the time 'cont' reaches a number multiple of 1000000
             print("---> ", datetime.datetime.now(), " - ", cont)
        
        i = 0
        
        if (cuiy[0] == row[0] and cuix[0] == row[1]): # We check we have a match between each diseases in both queries
            for metric in sim_metrics:
                files[i].write(", " + str(row[i+2]))
                i += 1 # Counter to get the value of each similarity metric in the right file
            
            row = crs.fetchone()
            
            if row == None:
                row = ("","")
        
        else:
            if cuiy[0] == cuix[0]: # In case we are in the diagonal
                x = 1
            else: # In case there is not similarity coefficient of the disease combination
                x = 0
            for metric in sim_metrics:
                files[i].write(", " + str(x))
                i += 1
    
    for file in files:
        file.write("\n")

for file in files:
    file.close() # Closing all the files
     
print(datetime.datetime.now()) # End time

print('DONE!')

2019-06-09 17:35:15.526931
--->  2019-06-09 17:35:49.192618  -  1000000
--->  2019-06-09 17:36:24.692189  -  2000000
--->  2019-06-09 17:36:59.935645  -  3000000
--->  2019-06-09 17:37:33.088889  -  4000000
--->  2019-06-09 17:38:06.056386  -  5000000
--->  2019-06-09 17:38:38.417066  -  6000000
--->  2019-06-09 17:39:10.235721  -  7000000
--->  2019-06-09 17:39:42.144839  -  8000000
--->  2019-06-09 17:40:14.055227  -  9000000
--->  2019-06-09 17:40:46.331309  -  10000000
--->  2019-06-09 17:41:18.236068  -  11000000
--->  2019-06-09 17:41:51.760149  -  12000000
--->  2019-06-09 17:42:23.799289  -  13000000
2019-06-09 17:42:39.578452
DONE!


In [7]:
# Closing the connection to the database when finished
cnx.close()